In [1]:
# library imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import json

from PIL import Image
Image.MAX_IMAGE_PIXELS = 120000000

import logging
logging.basicConfig(level=logging.INFO)

# local imports
from resnet_helper_functions import create_datasets, create_dataloaders, train_model

# torch parameters being used
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

torch:  1.10 ; cuda:  cu113


In [2]:
# read in config file
with open('resnet_config.json') as f:
    config = json.load(f)
    logging.info('Resnet Config File Read Successfully.')

# create datasets and dataloaders
train_dataset, val_dataset, test_dataset, class_names, num_classes = create_datasets(config["data_dir"], config["train_perc"], config["val_perc"], config["test_perc"])
logging.info('Train, Validation and Test Datasets Created Successfully.')

dataloaders, dataset_sizes = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config["batch_size"], num_workers = config["num_workers"])
logging.info('Train, Validation and Test Dataloaders Created Successfully.')

# instantiate pre-trained resnet
#net = torch.hub.load('pytorch/vision', config["pretrained_model_to_use"], weights=config["weights_to_use"])
net = torchvision.models.resnet18(pretrained=True)
logging.info('Model Loaded Successfully.')

# shut off autograd for all layers to freeze model so the layer weights are not trained
if config["freeze_pretrained_model"]:
    for param in net.parameters():
        param.requires_grad = False

# get the number of inputs to final FC layer
num_ftrs = net.fc.in_features

# replace existing FC layer with a new FC layer having the same number of inputs and num_classes outputs
net.fc = nn.Linear(num_ftrs, num_classes)

# cross entropy loss combines softmax and nn.NLLLoss() in one single class.
criterion = nn.CrossEntropyLoss()

# define optimizer
optimizer = optim.Adam(net.fc.parameters(), lr=0.001)

# learning rate scheduler - not using as we used adam optimizer
lr_scheduler = None

# set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logging.info(f'Device {device} Being Used.')

# train the model
model_dir = config["model_dir"]
num_epochs = config["num_epochs"]

logging.info('Started Training The Model.')

INFO:root:Resnet Config File Read Successfully.
INFO:root:Train, Validation and Test Datasets Created Successfully.
INFO:root:Train, Validation and Test Dataloaders Created Successfully.
INFO:root:Model Loaded Successfully.
INFO:root:Device cuda:0 Being Used.
INFO:root:Started Training The Model.


In [9]:
dataloaders['train']

In [5]:
for inputs, labels in dataloaders['train']:
    print(inputs, labels)
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/hpc/group/rescomp/sg623/miniconda3/envs/cv_540/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/hpc/group/rescomp/sg623/miniconda3/envs/cv_540/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/hpc/group/rescomp/sg623/miniconda3/envs/cv_540/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/hpc/group/rescomp/sg623/miniconda3/envs/cv_540/lib/python3.8/site-packages/torch/utils/data/dataset.py", line 363, in __getitem__
    return self.dataset[self.indices[idx]]
  File "/hpc/group/rescomp/sg623/miniconda3/envs/cv_540/lib/python3.8/site-packages/torchvision/datasets/folder.py", line 234, 

In [ ]:
net = train_model(model = net, model_dir = model_dir, criterion = criterion, optimizer = optimizer, dataloaders = dataloaders, dataset_sizes = dataset_sizes, scheduler = lr_scheduler, device = device, num_epochs = num_epochs)

# test the model
#test_model(model = net, test_dataset = test_dataset, device = device)

if __name__ == '__main__':
run_script()

In [11]:
x = "abc"

In [13]:
x.capitalize()

'Abc'